## Process
- Load MSA file
- Clean data and first mapping for bdd injection
- Drop duplicates
- Add default column values
- Output to CSV

## Encoding
MSA -> utf-8-sig, check for 2025

In [ ]:
import csv
import os
import pandas as pd
import json
import numpy as np

from dotenv import load_dotenv
from datetime import timedelta, datetime
from utils.data_utils import format_insee_or_postal_code, unaccent_and_upper

load_dotenv()

msa_filepath = os.environ['MSA_PATHFILE']
base_output_filepath = os.environ['DB_MSA_EXPORT']
exercice_id = 5

In [ ]:
msa_names = [
    'numero_allocataire',
    'organisme',
    'qualite_allocataire',
    'nom_allocataire',
    'prenom_allocataire',
    'commune_de_naissance',
    'code_insee_commune_alloc',
    'pays_de_naissance',
    'code_iso_pays',
    'date_de_naissance_alloc',
    'adresse_de_messagerie',
    'numero_tel_portable',
    'qualite_destinataire',
    'nom_destinataire',
    'prenom_destinataire',
    'complement_adresse',
    'numero_voie',
    'complement_numero_voie',
    'type_voie',
    'voie',
    'code_postal',
    'nom_commune',
    'code_insee_commune',
    'nom_beneficiaire',
    'prenom_beneficiaire',
    'genre_beneficiaire',
    'date_de_naissance_beneficiaire'
]

msa_column_type = {
    'code_postal': 'str',
    'numero_tel_portable': 'str',
    'code_insee_commune': 'str',
    'date_de_naissance_alloc': 'str',
    'date_de_naissance_beneficiaire': 'str',
    'numero_allocataire': 'str',
    'organisme': 'str',
    'genre_beneficiaire': 'str',
}

# todo: check encoding
msa_df = pd.read_csv(msa_filepath, encoding='ISO-8859-1', on_bad_lines='skip', sep=';', quoting=csv.QUOTE_NONE,
                     engine="c", names=msa_names, skiprows=1, header=None, dtype=msa_column_type)

In [ ]:
msa_column_mapping = {
    # infos allocataire
    'numero_allocataire': 'allocataire-matricule',
    'organisme': 'allocataire-code_organisme',
    'qualite_allocataire': 'allocataire-qualite',
    'nom_allocataire': 'allocataire-nom',
    'prenom_allocataire': 'allocataire-prenom',
    'date_de_naissance_alloc': 'allocataire-date_naissance',
    'pays_de_naissance': 'allocataire-pays_naissance',
    'code_insee_commune_alloc': 'allocataire-code_insee_commune_naissance',
    'commune_de_naissance': 'allocataire-commune_naissance',
    'code_iso_pays': 'allocataire-code_iso_pays_naissance',
    'adresse_de_messagerie': 'allocataire-courriel',
    'numero_tel_portable': 'allocataire-telephone',

    # adresse allocataire
    # pas de voie
    'nom_commune': 'adresse_allocataire-commune',
    'code_postal': 'adresse_allocataire-code_postal',
    'code_insee_commune': 'adresse_allocataire-code_insee',
    'complement_adresse': 'adresse_allocataire-cplt_adresse',

    # infos bénéficiaires
    'nom_beneficiaire': 'nom',
    'prenom_beneficiaire': 'prenom',
    'genre_beneficiaire': 'genre',
    'date_de_naissance_beneficiaire': 'date_naissance'
}

df_psp_mapped_msa = msa_df.copy()
df_psp_mapped_msa = df_psp_mapped_msa.rename(columns=msa_column_mapping)

In [ ]:
# trim all values
df_psp_mapped_msa['qualite_destinataire'] = df_psp_mapped_msa['qualite_destinataire'].str.strip()
df_psp_mapped_msa['nom_destinataire'] = df_psp_mapped_msa['nom_destinataire'].str.strip()
df_psp_mapped_msa['prenom_destinataire'] = df_psp_mapped_msa['prenom_destinataire'].str.strip()

In [ ]:

df_psp_mapped_msa['adresse_allocataire-commune'] = df_psp_mapped_msa['adresse_allocataire-commune'].str.strip()
df_psp_mapped_msa['adresse_allocataire-code_postal'] = format_insee_or_postal_code(
    df_psp_mapped_msa['adresse_allocataire-code_postal'].str.strip())
df_psp_mapped_msa['adresse_allocataire-code_insee'] = format_insee_or_postal_code(
    df_psp_mapped_msa['adresse_allocataire-code_insee'].str.strip())
df_psp_mapped_msa['adresse_allocataire-cplt_adresse'] = df_psp_mapped_msa[
    'adresse_allocataire-cplt_adresse'].str.strip()

In [ ]:
# gender
df_psp_mapped_msa['genre'] = df_psp_mapped_msa['genre'].replace({ '1': 'M', '2': 'F'})

# qualite allocataire
df_psp_mapped_msa['allocataire-qualite'] = df_psp_mapped_msa['allocataire-qualite'].str.strip().replace({'MME': 'Mme', 'MR': 'M'})

In [ ]:
# date de naissance allocataire
df_psp_mapped_msa['allocataire-date_naissance'] = df_psp_mapped_msa['allocataire-date_naissance'].replace(
    ['0', '00000000'], np.NaN)

df_psp_mapped_msa['allocataire-date_naissance'] = pd.to_datetime(df_psp_mapped_msa['allocataire-date_naissance'],
                                                                 format='%Y%m%d').dt.strftime('%d/%m/%Y')
# default situation "jeune", "AAH" is treated below
df_psp_mapped_msa['situation'] = 'jeune'
df_psp_mapped_msa['organisme'] = 'MSA'

In [ ]:
df_psp_mapped_msa['date_naissance'] = pd.to_datetime(df_psp_mapped_msa['date_naissance'], format='%Y%m%d')
# df_psp_mapped_msa['date_naissance'] = df_psp_mapped_msa['date_naissance'].dt.strftime('%d/%m/%Y')

# Benef between 20 and 30 years old are AAH & benef must have same names
mask_dob_start = df_psp_mapped_msa['date_naissance'].dt.date >= datetime(1995, 1, 1).date()
mask_dob_end = df_psp_mapped_msa['date_naissance'].dt.date <= datetime(2005, 12, 31).date()
mask_dob = mask_dob_start & mask_dob_end

mask_nom_equal = df_psp_mapped_msa['allocataire-nom'] == df_psp_mapped_msa['nom']
mask_prenom_equal = df_psp_mapped_msa['allocataire-prenom'] == df_psp_mapped_msa['prenom']
mask_same_names = mask_nom_equal & mask_prenom_equal

df_psp_mapped_msa.loc[mask_dob & mask_same_names, 'situation'] = 'AAH'

In [ ]:
## Street address
df_psp_mapped_msa['numero_voie'] = df_psp_mapped_msa['numero_voie'].str.strip()
df_psp_mapped_msa['complement_numero_voie'] = df_psp_mapped_msa['complement_numero_voie'].str.strip()
df_psp_mapped_msa['type_voie'] = df_psp_mapped_msa['type_voie'].str.strip()
df_psp_mapped_msa['voie'] = df_psp_mapped_msa['voie'].str.replace('"', '').str.strip()

In [ ]:
# Remove extra white spaces
df_psp_mapped_msa['nom'] = df_psp_mapped_msa['nom'].astype(str).apply(unaccent_and_upper).str.strip()
df_psp_mapped_msa['prenom'] = df_psp_mapped_msa['prenom'].astype(str).apply(unaccent_and_upper).str.strip()
df_psp_mapped_msa['genre'] = df_psp_mapped_msa['genre'].astype(str).str.upper()

In [ ]:
# Custom address name from allocataire
df_psp_mapped_msa['adresse_allocataire-nom_adresse_postale'] = df_psp_mapped_msa['qualite_destinataire'] + ' ' + \
                                                               df_psp_mapped_msa['nom_destinataire'] + ' ' + \
                                                               df_psp_mapped_msa['prenom_destinataire']

# Street address from allocataire
df_psp_mapped_msa['adresse_allocataire-voie'] = df_psp_mapped_msa['numero_voie'] + ' ' + df_psp_mapped_msa[
    'complement_numero_voie'] + ' ' + df_psp_mapped_msa['type_voie'] + df_psp_mapped_msa['voie']
df_psp_mapped_msa['adresse_allocataire-voie'] = df_psp_mapped_msa['adresse_allocataire-voie'].str.strip()

# zip code of where the allocataire was born
df_psp_mapped_msa['allocataire-code_insee_commune_naissance'] = format_insee_or_postal_code(
    df_psp_mapped_msa['allocataire-code_insee_commune_naissance']
)

In [ ]:
# Invalid phone number, set it to NaN to exclude them in LM mailing campaign
df_psp_mapped_msa['allocataire-telephone'] = df_psp_mapped_msa['allocataire-telephone'].replace(
    {'0000000000': np.NaN, '0600000000': np.NaN})

In [ ]:
# remove unused
df_psp_mapped_msa = df_psp_mapped_msa.drop(columns=[
    'nom_destinataire',
    'prenom_destinataire',
    'qualite_destinataire',
    'numero_voie',
    'complement_numero_voie',
    'type_voie',
    'voie'
])

In [ ]:
# trim sur tous les champs MSA (même dans les json)
df_psp_mapped_msa['allocataire-matricule'] = df_psp_mapped_msa['allocataire-matricule'].str.strip()
df_psp_mapped_msa['allocataire-qualite'] = df_psp_mapped_msa['allocataire-qualite'].str.strip()
df_psp_mapped_msa['allocataire-nom'] = df_psp_mapped_msa['allocataire-nom'].str.strip()
df_psp_mapped_msa['allocataire-prenom'] = df_psp_mapped_msa['allocataire-prenom'].str.strip()
df_psp_mapped_msa['allocataire-date_naissance'] = df_psp_mapped_msa['allocataire-date_naissance'].str.strip()
df_psp_mapped_msa['allocataire-pays_naissance'] = df_psp_mapped_msa['allocataire-pays_naissance'].str.strip()
df_psp_mapped_msa['allocataire-code_insee_commune_naissance'] = format_insee_or_postal_code(
    df_psp_mapped_msa['allocataire-code_insee_commune_naissance'].str.strip())
df_psp_mapped_msa['allocataire-commune_naissance'] = df_psp_mapped_msa['allocataire-commune_naissance'].str.strip()
df_psp_mapped_msa['allocataire-code_iso_pays_naissance'] = df_psp_mapped_msa[
    'allocataire-code_iso_pays_naissance'].str.strip()

df_psp_mapped_msa['allocataire-courriel'] = df_psp_mapped_msa['allocataire-courriel'].str.strip()
df_psp_mapped_msa['allocataire-telephone'] = df_psp_mapped_msa['allocataire-telephone'].str.strip()

In [ ]:
# concat into a single dataframe
df_all = pd.concat([df_psp_mapped_msa], axis=0, ignore_index=True)

# remove rows with missing necessary values (if one of those value are missing we cannot generate a code)
necessary_column = ['nom', 'prenom', 'date_naissance', 'genre']
df_all_valid_row = df_all.dropna(subset=necessary_column)

# remove columns with all null value
df_all_valid = df_all_valid_row.dropna(axis=1, how='all')

assert len(df_all_valid[
               df_all['nom'].isnull() | df_all_valid['prenom'].isnull() | df_all_valid['date_naissance'].isnull()]) == 0

In [ ]:
# lower case on emails on all
df_all_valid['allocataire-courriel'] = df_all_valid['allocataire-courriel'].str.lower()

In [ ]:
# remove rows when beneficiary is before september 1994
mask_before = pd.to_datetime(df_all_valid['date_naissance']) > datetime(1994, 9, 16)
df_all_valid = df_all_valid[mask_before]

print(f"{len(df_all_valid) - len(df_all_valid)} rows where removed because date_naissance was before 1994")

In [ ]:
# add missing 0 to phone numbers
mask_tel_not_null = df_all_valid['allocataire-telephone'].notna()
mask_no_zero_phone_number = ~df_all_valid.loc[mask_tel_not_null, 'allocataire-telephone'].str.startswith('0')
mask_9_char_phone = df_all_valid.loc[mask_tel_not_null, 'allocataire-telephone'].str.len() == 9
df_all_valid.loc[mask_tel_not_null & mask_no_zero_phone_number & mask_9_char_phone, 'allocataire-telephone'] = '0' + \
                                                                                                                df_all_valid[
                                                                                                                    'allocataire-telephone']

# set '0' phone values to None
mask_tel_eq_zero = df_all_valid['allocataire-telephone'] == '0'
df_all_valid.loc[mask_tel_eq_zero, 'allocataire-telephone'] = np.NaN

In [ ]:
# add 4h on all birthdates
df_all_valid['date_naissance'] = df_all_valid['date_naissance'] + timedelta(hours=4)

In [ ]:
# remove duplicate beneficiaries
df_all_valid_no_duplicate = df_all_valid.drop_duplicates(subset=[
    'date_naissance',
    'nom',
    'prenom',
    'genre',
    'organisme',
    'situation',
    'allocataire-qualite',
    'allocataire-matricule',
    'allocataire-code_organisme',
    'allocataire-telephone',

    # 'allocataire-nom',
    'allocataire-prenom',
    'allocataire-date_naissance',
    'allocataire-courriel',

    # we can remove this column additionaly
    'allocataire-code_insee_commune_naissance',
    'allocataire-commune_naissance',
    'allocataire-code_iso_pays_naissance',
    'allocataire-pays_naissance'
])

print(f"{len(df_all_valid) - len(df_all_valid_no_duplicate)} duplicate rows were removed")

In [ ]:
# map to json values for target DB model 
## map allocataire json
def to_json_allocataire_without_null(row):
    allocataire_mapping = {
        'qualite': row['allocataire-qualite'],
        'matricule': row['allocataire-matricule'],
        'code_organisme': row['allocataire-code_organisme'],
        'telephone': row['allocataire-telephone'],
        'nom': unaccent_and_upper(row['allocataire-nom']),
        'prenom': unaccent_and_upper(row['allocataire-prenom']),
        'date_naissance': row['allocataire-date_naissance'],
        'courriel': row['allocataire-courriel'],
        'code_insee_commune_naissance': row['allocataire-code_insee_commune_naissance'],
        'commune_naissance': row['allocataire-commune_naissance'],
        'code_iso_pays_naissance': row['allocataire-code_iso_pays_naissance'],
        'pays_naissance': row['allocataire-pays_naissance']
    }
    filtered_NaN_allocataire = {k: v for k, v in allocataire_mapping.items() if pd.notnull(v)}
    return json.dumps(filtered_NaN_allocataire, ensure_ascii=False)

df_all_valid_no_duplicate['allocataire'] = df_all_valid_no_duplicate.apply(to_json_allocataire_without_null, axis=1)

In [ ]:
## map adresse_allocataire json
def to_json_adresse_without_null(row):
    adresse_mapping = {
        'voie': row['adresse_allocataire-voie'],
        'code_postal': format_insee_or_postal_code(row['adresse_allocataire-code_postal']),
        'nom_adresse_postale': row['adresse_allocataire-nom_adresse_postale'],
        'commune': row['adresse_allocataire-commune'],
        'code_insee': format_insee_or_postal_code(row['adresse_allocataire-code_insee']),
        'cplt_adresse': row['adresse_allocataire-cplt_adresse'],
    }

    filtered_address = {k: v for k, v in adresse_mapping.items() if pd.notnull(v)}
    return json.dumps(filtered_address, ensure_ascii=False)


df_all_valid_no_duplicate['adresse_allocataire'] = df_all_valid_no_duplicate.apply(to_json_adresse_without_null, axis=1)

In [ ]:
## drop null value
df_final = df_all_valid_no_duplicate.drop(columns=[
    'allocataire-qualite',
    'allocataire-matricule',
    'allocataire-code_organisme',
    'allocataire-nom',
    'allocataire-prenom',
    'allocataire-telephone',
    'allocataire-date_naissance',
    'allocataire-courriel',
    'allocataire-code_insee_commune_naissance',
    'allocataire-commune_naissance',
    'allocataire-code_iso_pays_naissance',
    'allocataire-pays_naissance',
    'adresse_allocataire-voie',
    'adresse_allocataire-nom_adresse_postale',
    'adresse_allocataire-code_postal',
    'adresse_allocataire-commune',
    'adresse_allocataire-code_insee',
    'adresse_allocataire-cplt_adresse',
])


In [ ]:
# Add missing default column needed for target DB model
df_final['updated_at'] = datetime.now()
df_final['exercice_id'] = exercice_id
df_final['uuid_doc'] = np.NaN

df_final['updated_at'] = df_final['updated_at'].astype(str)
df_final['date_naissance'] = df_final['date_naissance'].astype(str)

In [ ]:
# output to CSV
df_final.to_csv(base_output_filepath, sep=';', index=False, encoding='utf-8')